Feature 1: This is a data set I created for a fictional Fruit and Veggie sales company. They want to know their top most profitable customers and their most profitable regions. Their data is spread out between three files. In the below code block the three files are read into Python, the excel file is reformated to .csv and they are merged into one master file. This file is called 'comp_data.csv' and is saved in the 'final_files' folder.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cu_data = pd.read_csv('final_files/cust_data.csv')
or_data = pd.read_csv('final_files/order_data.csv')
prd_data = pd.read_excel('final_files/prod_data.xlsx')

cust_order_data = pd.merge(cu_data, or_data, on='customer_id', how='outer')

comp_data = pd.merge(prd_data, cust_order_data, on='product_id', how='outer')

comp_data.to_csv('final_files/comp_data.csv', sep=',', index=False)


#Use the below code if you want to review the combine file. 

#comp_data_set = pd.read_csv('final_files/comp_data.csv')
#print(comp_data_set.to_string())



Feature 2: Prior to analyzng the data it needs to be cleaned. In the below code block unwanted columns are removed, cells with NaN values are updated, and data types are corrected so figures can be correctly calulated in the later analysis.

In [ ]:
col_drop = pd.read_csv('final_files/comp_data.csv')
col_drop1 = pd.DataFrame(col_drop)
col_drop2 = col_drop1.drop(['product_type', 'years_served'], axis=1, inplace=False)
col_org = col_drop2.sort_values(['order_date', 'order_id'], inplace=False)

col_nan = col_org.fillna({'product_id':'blank','product_name':'blank', 
                          'unit_price':'0.00', 'unit_cost':'0.00', 'order_id':'blank', 'quantity':'0.0', 
                          'order_date':'0/00/0000', 'order_status':'blank'}, inplace=False)

col_type = col_nan.astype({'unit_price':'float', 'unit_cost':'float', 'customer_id':'object', 'quantity':'float'})
print(col_type.to_string())

Feature 3: The customer wants to know the top 15 customers by the profit and which region is most profitable. The coding below creates a new column called 'Profits' based off of the Unit Price, Unit Cost, and Quantity sold. The sales are then grouped by each customer and the print show spreadsheet format top 15 customers. The second block of coding show the regions listed in order of the most profitable to least profitable.

In [ ]:
#Code to show the top 15 most profitable customers.
col_type['profit'] = ((col_type['unit_price'] - col_type['unit_cost']) * col_type['quantity'])
col_group = col_type.groupby(['customer_id'], sort=True)['profit'].sum().reset_index()
col_group = col_group.sort_values(by = ['profit'], ascending=[False]).head(15)
top_cust = pd.merge(col_group, cu_data, on='customer_id', how='left')

#Code to Show the regions by most profitable.
col_type['profit'] = ((col_type['unit_price'] - col_type['unit_cost']) * col_type['quantity'])
col_group = col_type.groupby(['region'], sort=True)['profit'].sum().reset_index()
col_group = col_group.sort_values(by = ['profit'], ascending=[False]).head()

print(top_cust.to_string())
print(col_group.to_string())


Feature 4: The Customer not only wanted the data available in text formatting they also wanted to see a graphical showing of the data. Below is the coding that show teh top customers and regions in graphs.

In [ ]:
#This code block is for the top 15 customers.
plot1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
plot1.set_title('Profits By Customer')
plot1.set_xlabel('Customer')
plot1.set_ylabel('Profits in Dollars')

#This code block is for the regions listed by most profitable.
plot2 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
plot2.set_title('Profits by Sales Region')
plot2.set_xlabel('Profits')
plot2.set_ylabel('Sales per Regions')


plot1.bar(top_cust['customer_name_first'], top_cust['profit'])
plot2.plot(col_group['region'], col_group['profit'])

plt.tight_layout()
plt.show()
